In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
import numpy as np
import cv2

def nothing(x):
    pass

cv2.namedWindow("tracking")
cv2.createTrackbar("LH","tracking",0,255,nothing)
cv2.createTrackbar("LS","tracking",0,255,nothing)
cv2.createTrackbar("LV","tracking",0,255,nothing)
cv2.createTrackbar("UH","tracking",255,255,nothing)
cv2.createTrackbar("US","tracking",255,255,nothing)
cv2.createTrackbar("UV","tracking",255,255,nothing)

# my_img = cv2.imread('First-CT-scan-brain-without-contrast-showing-suspicion-of-subarachnoid-bleed-a.png')

while True:

    my_img = cv2.imread('An-81-year-old-man-with-a-well-circumscribed-right-parietal-intraparenchymal-hematoma_Q320.jpg')
    #roi = img[269: 795, 537: 1416]
    rows, cols, _ = my_img.shape
    gray_roi = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)
    gray_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)
    _, threshold = cv2.threshold(gray_roi, 3, 255, cv2.THRESH_BINARY_INV)

    hsv = cv2.cvtColor(my_img, cv2.COLOR_BGR2HSV)

    """
    lh = 0
    ls = 0
    lv = 128
    uh = 255
    us = 255
    uv = 148 or 186 or 215
    """

    l_h = cv2.getTrackbarPos("LH" , "tracking")
    l_s = cv2.getTrackbarPos("LS", "tracking")
    l_v = cv2.getTrackbarPos("LV", "tracking")

    u_h = cv2.getTrackbarPos("UH", "tracking")
    u_s = cv2.getTrackbarPos("US", "tracking")
    u_v = cv2.getTrackbarPos("UV", "tracking")

    l_b = np.array([l_h,l_s,l_v])
    u_b = np.array([u_h,u_s,u_v])

    mask = cv2.inRange(hsv, l_b, u_b)
    res = cv2.bitwise_and(my_img,my_img,mask=mask)
    res2 = cv2.bitwise_and(gray_roi, gray_roi, mask=mask)
    
    #ret, thresh = cv2.threshold(my_img, 127, 255, 0)
    ret, thresh = cv2.threshold(my_img, 120, 200, 0)
    contours, hierarchy = cv2.findContours(res2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        #cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
        cv2.rectangle(my_img,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.line(my_img, (x + int(w/2), 0), (x + int(w/2), rows), (0, 255, 0), 2)
        cv2.line(my_img, (0, y + int(h/2)), (cols, y + int(h/2)), (0, 255, 0), 2)
        break


    cv2.imshow('Video', my_img)
    cv2.imshow('mask' , mask)
    cv2.imshow("res" , res2)
    if (cv2.waitKey(10) & 0xFF == ord('b')):
        break

KeyboardInterrupt: 